In [1]:
import os
import warnings
import findfile
import pandas as pd
import torch
import transformers

c:\Users\hongk\Documents\GitHub\ta-final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv-absa-3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(transformers.__version__)

4.28.0


In [3]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4060 Ti'

In [4]:
pip install datasets


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\hongk\AppData\Local\Programs\Python\Python39\lib\threading.py:921: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  del self._target, self._args, self._kwargs
C:\Users\hongk\AppData\Local\Programs\Python\Python39\lib\threading.py:921: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  del self._target, self._args, self._kwargs
C:\Users\hongk\AppData\Local\Programs\Python\Python39\lib\threading.py:921: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  del self._target, self._args, self._kwargs


In [5]:
from pyabsa import ABSAInstruction as absa_instruction

Version 0.9.10.post1 of metric_visualizer is outdated. Version 0.9.13.post1 was released Wednesday May 01, 2024.
[2024-07-05 18:17:10] (2.4.1.post1) PyABSA(2.4.1.post1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






c:\Users\hongk\Documents\GitHub\ta-final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv-absa-3.9\lib\site-packages\tensorflow\python\framework\dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
c:\Users\hongk\Documents\GitHub\ta-final-project-aspect-based-sentiment-analysis-and-categorized-aspect\venv-absa-3.9\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
C:\Users\hongk\AppData\Local\Programs\Python\Python39\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


In [6]:
warnings.filterwarnings("ignore")

In [7]:
task_name = "multitask"
experiment_name = "quadruple-ss-less-cat-05-07-24"
# model_checkpoint = "google/flan-t5-base"
model_checkpoint = "kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined"

In [8]:
print("Experiment Name: ", experiment_name)
model_out_path = "checkpoints"
model_out_path = os.path.join(
    model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}"
)
print("Model output path: ", model_out_path)

Experiment Name:  quadruple-ss-less-cat-05-07-24
Model output path:  checkpoints\multitask\kevinscariaate_tk-instruct-base-def-pos-neg-neut-combined-quadruple-ss-less-cat-05-07-24


In [9]:
id_train_file_path = "./ABSADatasets/datasets/acos_datasets/5000.Hotel-ss-less-cat"
id_test_file_path = "./ABSADatasets/datasets/acos_datasets/5000.Hotel-ss-less-cat"

In [10]:
import json

In [12]:
id_tr_df = absa_instruction.data_utils.read_json(id_train_file_path,"train")
id_te_df = absa_instruction.data_utils.read_json(id_test_file_path, "test")

./ABSADatasets/datasets/acos_datasets/5000.Hotel-ss-less-cat\train.jsonl
./ABSADatasets/datasets/acos_datasets/5000.Hotel-ss-less-cat\test.jsonl


In [13]:
id_tr_df

[{'text': 'Beautiful , comfortable and friendly hotel near the beach .',
  'labels': [{'aspect': 'hotel',
    'opinion': 'friendly',
    'polarity': 'POS',
    'category': 'SERVICE#GENERAL'},
   {'aspect': 'hotel',
    'opinion': 'comfortable',
    'polarity': 'POS',
    'category': 'EXPERIENCE,AMBIENCE'},
   {'aspect': 'hotel',
    'opinion': 'Beautiful',
    'polarity': 'POS',
    'category': 'EXPERIENCE,AMBIENCE'}]},
 {'text': 'Great value for the money .',
  'labels': [{'aspect': 'money',
    'opinion': 'Great value',
    'polarity': 'POS',
    'category': 'HOTEL#PRICE'}]},
 {'text': 'Good situation , next to a big mall and the beach .',
  'labels': [{'aspect': 'beach',
    'opinion': 'next to',
    'polarity': 'POS',
    'category': 'LOCATION#ATTRACTIONS'},
   {'aspect': 'mall',
    'opinion': 'next to',
    'polarity': 'POS',
    'category': 'LOCATION#ENTERTAINMENT'}]},
 {'text': 'Rooms really comfortable , clean and no noisy .',
  'labels': [{'aspect': 'Rooms',
    'opinion': 'c

In [14]:
id_tr_df = pd.DataFrame(id_tr_df)
id_te_df = pd.DataFrame(id_te_df)

In [15]:
loader = absa_instruction.data_utils.InstructDatasetLoader(id_tr_df, id_te_df)

In [16]:
if loader.train_df_id is not None:
    loader.train_df_id = loader.prepare_instruction_dataloader(loader.train_df_id)
if loader.test_df_id is not None:
    loader.test_df_id = loader.prepare_instruction_dataloader(loader.test_df_id)
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.prepare_instruction_dataloader(loader.train_df_ood)
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.prepare_instruction_dataloader(loader.test_df_ood)

In [17]:
# Create T5 utils object
t5_exp = absa_instruction.model.T5Generator(model_checkpoint)

[2024-07-05 18:18:21] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-07-05 18:18:21] (2.4.1.post1) ********** Available ACOS model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-07-05 18:18:21] (2.4.1.post1) Checkpoint:kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined is not found, you can raise an issue for requesting shares of checkpoints
[2024-07-05 18:18:21] (2.4.1.post1) No checkpoint found in Model Hub for task: kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined


In [18]:
# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.create_datasets(
    t5_exp.tokenize_function_inputs
)

Map: 100%|██████████| 1604/1604 [00:00<00:00, 13114.74 examples/s]


In [19]:
training_args = {
    "output_dir": model_out_path,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 3,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "load_best_model_at_end": True,
    "push_to_hub": False,
    "eval_accumulation_steps": 1,
    "predict_with_generate": True,
    "logging_steps": 1000000000,
    "use_mps_device": False,
    # 'fp16': True,
    "fp16": False,
}


In [20]:
# Train model
model_trainer_less_category = t5_exp.train(id_tokenized_ds, **training_args)

Trainer device: cuda:0

Model training started ....


 33%|███▎      | 5676/17028 [15:40<30:11,  6.27it/s]

{'eval_loss': 0.18881845474243164, 'eval_runtime': 15.3234, 'eval_samples_per_second': 104.677, 'eval_steps_per_second': 6.591, 'epoch': 1.0}


                                                       
 67%|██████▋   | 11352/17028 [31:27<16:16,  5.81it/s]

{'eval_loss': 0.15111969411373138, 'eval_runtime': 15.4158, 'eval_samples_per_second': 104.049, 'eval_steps_per_second': 6.552, 'epoch': 2.0}


                                                       
100%|██████████| 17028/17028 [47:22<00:00,  6.00it/s]

{'eval_loss': 0.14707931876182556, 'eval_runtime': 15.6677, 'eval_samples_per_second': 102.376, 'eval_steps_per_second': 6.446, 'epoch': 3.0}


100%|██████████| 17028/17028 [47:25<00:00,  5.98it/s]


{'train_runtime': 2845.5172, 'train_samples_per_second': 23.937, 'train_steps_per_second': 5.984, 'train_loss': 0.3346473168733116, 'epoch': 3.0}


In [22]:
# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(
    predictor= model_trainer_less_category,
    tokenized_dataset=id_tokenized_ds,
    sample_set="train",
    batch_size=16,
)
id_tr_labels = [i.strip() for i in id_ds["train"]["labels"]]

Prediction from trainer


100%|██████████| 1419/1419 [15:49<00:00,  1.49it/s]


In [23]:
# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(
    predictor = model_trainer_less_category,
    tokenized_dataset = id_tokenized_ds,
    sample_set = "test",
    batch_size = 16,
)
id_te_labels = [i.strip() for i in id_ds["test"]["labels"]]

Prediction from trainer


100%|██████████| 101/101 [01:01<00:00,  1.63it/s]


In [32]:
import numpy as np

# Assuming id_tr_pred_labels and id_te_pred_labels are lists of strings
# Convert them to numpy arrays
id_tr_pred_labels_np = np.array(id_tr_pred_labels)
id_tr_labels_np = np.array(id_tr_labels)
id_te_pred_labels_np = np.array(id_te_pred_labels)
id_te_labels_np = np.array(id_te_labels)

# Save the arrays to a .npz file
np.savez('prediction_labels.npz', 
         id_tr_pred_labels=id_tr_pred_labels_np, 
         id_tr_labels=id_tr_labels_np, 
         id_te_pred_labels=id_te_pred_labels_np, 
         id_te_labels=id_te_labels_np)

In [33]:
# Load the arrays from the .npz file
loaded_data = np.load('prediction_labels.npz')

# Access the arrays
id_tr_pred_labels_loaded = loaded_data['id_tr_pred_labels']
id_tr_labels_loaded = loaded_data['id_tr_labels']
id_te_pred_labels_loaded = loaded_data['id_te_pred_labels']
id_te_labels_loaded = loaded_data['id_te_labels']

# Convert back to lists if needed
id_tr_pred_labels_list = id_tr_pred_labels_loaded.tolist()
id_tr_labels_list = id_tr_labels_loaded.tolist()
id_te_pred_labels_list = id_te_pred_labels_loaded.tolist()
id_te_labels_list = id_te_labels_loaded.tolist()

In [37]:
print(id_tr_pred_labels_list)

['room', 'room::NEG', 'room::small|room::nosignal', 'room::EXPERIENCE,AMBIENCE|room::EXPERIENCE,AMBIENCE', 'swimmingpool', 'swimmingpool:NEG|look:NEU', 'swimmingpool:notuse|look:ok', 'swimmingpool:FACILITY#POOL|look:EXPERIENCE,AMBIENCE', 'poolfacilities', 'poolfacilities:NEG', 'poolfacilities:didnotgettouse', 'poolfacilities:FACILITY#POOL', 'Breakfast', 'Breakfast:NEG', 'Breakfast:crap', 'Breakfast:FOOD#TASTE', 'Deluxeroom|staff', 'room:POS|staff:POS', 'room:comfortable|staff:wonderful', 'room:EXPERIENCE,AMBIENCE|staff:SERVICE#GENERAL', 'breakfastoptions', 'breakfast:NEG', 'breakfast:limited', 'breakfast:FOOD#CHOICE', 'staffs', 'staffs:POS', 'staffs:helpful|staffs:Great', 'staffs:SERVICE#GENERAL|staffs:SERVICE#GENERAL', 'internetspeed', 'internetspeed:NEG|deal:POS', 'internetspeed:annoyed|deal:good', 'internetspeed:FACILITY#WIFI|deal:HOTEL#PRICE', 'swimmingpool', 'pool:POS', 'pool:large', 'pool:FACILITY#POOL', 'hotel|staff|pool|balcony', 'hotel:POS|staff:POS|poolinthegarden:POS|balcony

In [35]:
print(len(id_tr_pred_labels_list))
print(len(id_tr_labels_list))

22704
22704


In [36]:
print(len(id_te_pred_labels_list))
print(len(id_te_labels_list))

1604
1604


In [24]:
id_te_labels

['pool',
 'pool:POS',
 'pool:amazing',
 'pool:FACILITY#POOL',
 'staff',
 'staff:POS',
 'staff:friendly|staff:friendly',
 'staff:SERVICE#GENERAL|staff:SERVICE#GENERAL',
 'bathrooms',
 'bathrooms:POS',
 'bathrooms:good',
 'bathrooms:ROOM#BATHROOM',
 'internet',
 'internet:NEG',
 'internet:inconsistent',
 'internet:FACILITY#WIFI',
 'hotel',
 'hotel:POS',
 'hotel:perfect',
 'hotel:HOTEL#GENERAL',
 'staff',
 'staff:POS',
 'staff:lovely',
 'staff:SERVICE#GENERAL',
 'stay',
 'stay:POS',
 'stay:loved',
 'stay:EXPERIENCE,AMBIENCE',
 'rooms|balcony',
 'rooms:POS|balcony:POS',
 'rooms:large|rooms:clean|balcony:amazing',
 'rooms:ROOM#SPACE|rooms:ROOM#CLEANLINESS|balcony:ROOM#VIEW',
 'internet',
 'internet:POS',
 'internet:worked well',
 'internet:FACILITY#WIFI',
 'staff',
 'staff:POS',
 'staff:friendly',
 'staff:SERVICE#GENERAL',
 'room',
 'room:POS',
 'room:comfortable',
 'room:EXPERIENCE,AMBIENCE',
 'staff|pool',
 'staff:POS|pool:POS',
 'staff:friendly|pool:amazing',
 'staff:SERVICE#GENERAL|pool

In [39]:
id_tr_labels

['room:',
 'room::NEG',
 'room::small|room::nosignal',
 'room::ROOM#SPACE|room::FACILITY#WIFI',
 'swimmingpool|look',
 'swimmingpool:NEG|look:POS|swimmingpool:POS',
 'swimmingpool:notuse|look:ok|swimmingpool:ok',
 'swimmingpool:FACILITY#POOL|look:EXPERIENCE,AMBIENCE|swimmingpool:FACILITY#POOL',
 'poolfacilities',
 'poolfacilities:NEG',
 'poolfacilities:notgettouse',
 'poolfacilities:EXPERIENCE,AMBIENCE',
 'Breakfast',
 'Breakfast:NEG',
 'Breakfast:crap',
 'Breakfast:FOOD#TASTE',
 'room|staff',
 'room:POS|staff:POS',
 'room:comfortable|staff:wonderful',
 'room:EXPERIENCE,AMBIENCE|staff:SERVICE#GENERAL',
 'breakfast',
 'breakfast:NEG',
 'breakfast:optionswereabitlimited',
 'breakfast:FOOD#CHOICE',
 'staffs',
 'staffs:POS',
 'staffs:helpful|staffs:Great',
 'staffs:SERVICE#GENERAL|staffs:SERVICE#GENERAL',
 'internetspeed|deal',
 'internetspeed:NEG|deal:POS',
 'internetspeed:bitannoyed|deal:good',
 'internetspeed:FACILITY#WIFI|deal:HOTEL#PRICE',
 'pool',
 'pool:POS',
 'pool:large',
 'pool:F

In [31]:
metrics = t5_exp.get_classic_metrics(id_tr_labels, id_tr_pred_labels)
print("----------------------- Classic Training Set Metrics -----------------------")
print(metrics)

room:
room
room::NEG
room::NEG
room::small|room::nosignal
room::small|room::nosignal
room::ROOM#SPACE|room::FACILITY#WIFI
room::EXPERIENCE,AMBIENCE|room::EXPERIENCE,AMBIENCE
swimmingpool|look
swimmingpool
swimmingpool:NEG|look:POS|swimmingpool:POS
swimmingpool:NEG|look:NEU
swimmingpool:notuse|look:ok|swimmingpool:ok
swimmingpool:notuse|look:ok
swimmingpool:FACILITY#POOL|look:EXPERIENCE,AMBIENCE|swimmingpool:FACILITY#POOL
swimmingpool:FACILITY#POOL|look:EXPERIENCE,AMBIENCE
poolfacilities
poolfacilities
poolfacilities:NEG
poolfacilities:NEG
poolfacilities:notgettouse
poolfacilities:didnotgettouse
poolfacilities:EXPERIENCE,AMBIENCE
poolfacilities:FACILITY#POOL
Breakfast
Breakfast
Breakfast:NEG
Breakfast:NEG
Breakfast:crap
Breakfast:crap
Breakfast:FOOD#TASTE
Breakfast:FOOD#TASTE
room|staff
Deluxeroom|staff
room:POS|staff:POS
room:POS|staff:POS
room:comfortable|staff:wonderful
room:comfortable|staff:wonderful
room:EXPERIENCE,AMBIENCE|staff:SERVICE#GENERAL
room:EXPERIENCE,AMBIENCE|staff:SERV

NameError: name 'confusion_matrix' is not defined

In [29]:
metrics = t5_exp.get_classic_metrics(id_te_labels, id_te_pred_labels)
print("----------------------- Classic Testing Set Metrics -----------------------")
print(metrics)

pool
pool
pool:POS
pool:POS
pool:amazing
pool:amazing
pool:FACILITY#POOL
pool:FACILITY#POOL
staff
staff|cleaning|roomservice
staff:POS
staff:POS
staff:friendly|staff:friendly
staff:friendly
staff:SERVICE#GENERAL|staff:SERVICE#GENERAL
staff:SERVICE#GENERAL|staff:SERVICE#GENERAL
bathrooms
rooms|bathrooms
bathrooms:POS
bathrooms:POS
bathrooms:good
bathrooms:good
bathrooms:ROOM#BATHROOM
bathrooms:ROOM#BATHROOM
internet
internet
internet:NEG
internet:NEG
internet:inconsistent
internet:inconsistent
internet:FACILITY#WIFI
internet:FACILITY#WIFI
hotel
hotel
hotel:POS
hotel:POS
hotel:perfect
hotel:perfect
hotel:HOTEL#GENERAL
hotel:EXPERIENCE,AMBIENCE|hotel:EXPERIENCE,AMBIENCE
staff
hotel|staff
staff:POS
staff:POS
staff:lovely
staff:good|staff:lovely
staff:SERVICE#GENERAL
staff:SERVICE#GENERAL|staff:SERVICE#GENERAL
stay
stay
stay:POS
stay:POS
stay:loved
stay:loved
stay:EXPERIENCE,AMBIENCE
stay:EXPERIENCE,AMBIENCE
rooms|balcony
rooms|balcony
rooms:POS|balcony:POS
rooms:POS|balcony:POS
rooms:large

NameError: name 'confusion_matrix' is not defined

In [30]:
metrics

NameError: name 'metrics' is not defined

In [ ]:
id_te_labels = id_te_df['labels'] 
id_te_labels

0      [{'aspect': 'budget', 'opinion': 'nice', 'pola...
1      [{'aspect': 'Hotel', 'opinion': 'moved', 'pola...
2      [{'aspect': 'room', 'opinion': 'smelled', 'pol...
3      [{'aspect': 'duvet cover', 'opinion': 'stains'...
4      [{'aspect': 'wifi', 'opinion': 'appalling', 'p...
                             ...                        
296    [{'aspect': 'room size', 'opinion': 'Excellent...
297    [{'aspect': 'lift', 'opinion': 'No', 'polarity...
298    [{'aspect': 'priced', 'opinion': 'reasonably',...
299    [{'aspect': 'rooms', 'opinion': 'basic', 'pola...
300    [{'aspect': 'Staff', 'opinion': 'friendly', 'p...
Name: labels, Length: 301, dtype: object

In [38]:
id_te_pred_labels

['pool',
 'pool:POS',
 'pool:amazing',
 'pool:FACILITY#POOL',
 'staff|cleaning|roomservice',
 'staff:POS',
 'staff:friendly',
 'staff:SERVICE#GENERAL|staff:SERVICE#GENERAL',
 'rooms|bathrooms',
 'bathrooms:POS',
 'bathrooms:good',
 'bathrooms:ROOM#BATHROOM',
 'internet',
 'internet:NEG',
 'internet:inconsistent',
 'internet:FACILITY#WIFI',
 'hotel',
 'hotel:POS',
 'hotel:perfect',
 'hotel:EXPERIENCE,AMBIENCE|hotel:EXPERIENCE,AMBIENCE',
 'hotel|staff',
 'staff:POS',
 'staff:good|staff:lovely',
 'staff:SERVICE#GENERAL|staff:SERVICE#GENERAL',
 'stay',
 'stay:POS',
 'stay:loved',
 'stay:EXPERIENCE,AMBIENCE',
 'rooms|balcony',
 'rooms:POS|balcony:POS',
 'rooms:large|rooms:clean|balcony:amazing',
 'rooms:ROOM#SPACE|rooms:ROOM#CLEANLINESS|balcony:ROOM#VIEW',
 'internet',
 'internet:POS',
 'internet:workedwell',
 'internet:FACILITY#WIFI',
 'staff',
 'staff:POS',
 'staff:friendly',
 'staff:SERVICE#GENERAL',
 'room',
 'room:POS',
 'room:comfortable',
 'room:EXPERIENCE,AMBIENCE',
 'staff|pool',
 

In [ ]:
import pandas as pd

# Extract text and correct labels from the test dataset
id_te_texts = id_te_df['text']  # Assuming 'text' is the column name for the review text
id_te_correct_labels = id_te_df['labels'] 
print(id_te_labels)
print(id_te_texts)

output_df = pd.DataFrame({
    'text': id_te_texts,
    # 'predicted_labels': id_te_pred_labels,
    'correct_labels': id_te_correct_labels
})

output_df.head()


0      [{'aspect': 'budget', 'opinion': 'nice', 'pola...
1      [{'aspect': 'Hotel', 'opinion': 'moved', 'pola...
2      [{'aspect': 'room', 'opinion': 'smelled', 'pol...
3      [{'aspect': 'duvet cover', 'opinion': 'stains'...
4      [{'aspect': 'wifi', 'opinion': 'appalling', 'p...
                             ...                        
296    [{'aspect': 'room size', 'opinion': 'Excellent...
297    [{'aspect': 'lift', 'opinion': 'No', 'polarity...
298    [{'aspect': 'priced', 'opinion': 'reasonably',...
299    [{'aspect': 'rooms', 'opinion': 'basic', 'pola...
300    [{'aspect': 'Staff', 'opinion': 'friendly', 'p...
Name: labels, Length: 301, dtype: object
0                  but nice hotel for budget traveller .
1      We moved to this Hotel because its closer to t...
2      The room smelled very musty , it was unclean a...
3      My duvet cover had some brown undefined stains...
4      The wifi was appalling , the pool area was com...
                             ...               

,text,correct_labels
0,but nice hotel for budget traveller .,"[{'aspect': 'budget', 'opinion': 'nice', 'pola..."
1,We moved to this Hotel because its closer to t...,"[{'aspect': 'Hotel', 'opinion': 'moved', 'pola..."
2,"The room smelled very musty , it was unclean a...","[{'aspect': 'room', 'opinion': 'smelled', 'pol..."
3,My duvet cover had some brown undefined stains...,"[{'aspect': 'duvet cover', 'opinion': 'stains'..."
4,"The wifi was appalling , the pool area was com...","[{'aspect': 'wifi', 'opinion': 'appalling', 'p..."
